# 光学薄膜优化程序

author: Jiang anqing

## 导入必要的包:

* common.TransferMatrix    光学平面薄膜仿真程序
* common.FileEnviroment    平面膜强化学习环境
* common.FilmLoss          薄膜Loss计算


In [2]:
from common.DataLoader import MaterialLoader
from common.TransferMatrix import OpticalModeling
from common.Config import FilmConfig
from common.utils.FilmLoss import film_loss

['C:\\Users\\anqin\\AppData\\Local\\Temp\\3ea217aa-27f7-4032-b8ae-640db27960b3', 'c:\\Users\\anqin\\.vscode\\extensions\\ms-python.python-2020.2.64397\\pythonFiles', 'c:\\Users\\anqin\\.vscode\\extensions\\ms-python.python-2020.2.64397\\pythonFiles\\lib\\python', 'D:\\anaconda\\envs\\cuenv\\python37.zip', 'D:\\anaconda\\envs\\cuenv\\DLLs', 'D:\\anaconda\\envs\\cuenv\\lib', 'D:\\anaconda\\envs\\cuenv', '', 'C:\\Users\\anqin\\AppData\\Roaming\\Python\\Python37\\site-packages', 'D:\\anaconda\\envs\\cuenv\\lib\\site-packages', 'D:\\anaconda\\envs\\cuenv\\lib\\site-packages\\win32', 'D:\\anaconda\\envs\\cuenv\\lib\\site-packages\\win32\\lib', 'D:\\anaconda\\envs\\cuenv\\lib\\site-packages\\Pythonwin', 'D:\\anaconda\\envs\\cuenv\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\anqin\\.ipython', '..', 'd:\\MyCode\\filmmaker\\']


## 导入Python第三方必要的包
* numpy
* pandas
* tensroflow
* tensorflow-agents

In [6]:

import sys
sys.path.append('./agents/')

from tf_agents.agents.dqn import dqn_agent
from tf_agents.drivers import dynamic_step_driver
from tf_agents.eval import metric_utils
from tf_agents.metrics import tf_metrics
from tf_agents.networks import q_network
from tf_agents.policies import random_tf_policy
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.trajectories import trajectory
from tf_agents.utils.common import element_wise_squared_loss
from tf_agents.environments import tf_py_environment



#from common.FilmEnvironment import FilmEnvironment 


In [7]:
import tensorflow as tf 
import numpy as np 

### 导入env

In [8]:
tf_env = tf_py_environment.TFPyEnvironment(FilmEnvironment(config_path='Zn.ini'))

time_step = tf_env.reset()
rewards = []
steps = []
num_episodes = 5

for _ in range(num_episodes):
  episode_reward = 0
  episode_steps = 0
  while not time_step.is_last():
    action = tf.random.uniform([1], 0, 2, dtype=tf.int32)
    time_step = tf_env.step(action)
    episode_steps += 1
    episode_reward += time_step.reward.numpy()
  rewards.append(episode_reward)
  steps.append(episode_steps)
  time_step = tf_env.reset()

num_steps = np.sum(steps)
avg_length = np.mean(steps)
avg_reward = np.mean(rewards)

print('num_episodes:', num_episodes, 'num_steps:', num_steps)
print('avg_length', avg_length, 'avg_reward:', avg_reward)

num_episodes: 5 num_steps: 5
avg_length 1.0 avg_reward: 0.1


In [5]:
filmEnv = FilmEnvironment(config_path='Zn.ini')

print('Action Spec:')
print(filmEnv.action_spec())
print('Observation Spec:')
print(filmEnv.time_step_spec().observation)
print('Reward Spec:')
print(filmEnv.action_spec())
print('Observation Spec:')
print(filmEnv.time_step_spec().observation)
print('Reward Spec:')
print(filmEnv.time_step_spec().reward)

filmEnv = tf_py_environment.TFPyEnvironment(filmEnv)

Action Spec:
BoundedArraySpec(shape=(1,), dtype=dtype('int64'), name='action', minimum=0, maximum=18)
Observation Spec:
BoundedArraySpec(shape=(1,), dtype=dtype('float32'), name='observation', minimum=0.0, maximum=3.4028234663852886e+38)
Reward Spec:
BoundedArraySpec(shape=(1,), dtype=dtype('int64'), name='action', minimum=0, maximum=18)
Observation Spec:
BoundedArraySpec(shape=(1,), dtype=dtype('float32'), name='observation', minimum=0.0, maximum=3.4028234663852886e+38)
Reward Spec:
ArraySpec(shape=(), dtype=dtype('float32'), name='reward')


In [4]:
num_iterations = 20000 # @param {type:"integer"}

initial_collect_steps = 1000  # @param {type:"integer"} 
collect_steps_per_iteration = 1  # @param {type:"integer"}
replay_buffer_max_length = 100000  # @param {type:"integer"}

batch_size = 64  # @param {type:"integer"}
learning_rate = 1e-3  # @param {type:"number"}
log_interval = 200  # @param {type:"integer"}

num_eval_episodes = 10  # @param {type:"integer"}
eval_interval = 1000  # @param {type:"integer"}

In [6]:
print('Action Spec:')
print(filmEnv.action_spec())
print('Observation Spec:')
print(filmEnv.time_step_spec().observation)
print('Reward Spec:')
print(filmEnv.action_spec())
print('Observation Spec:')
print(filmEnv.time_step_spec().observation)
print('Reward Spec:')
print(filmEnv.time_step_spec().reward)

Action Spec:
BoundedTensorSpec(shape=(1,), dtype=tf.int64, name='action', minimum=array(0, dtype=int64), maximum=array(18, dtype=int64))
Observation Spec:
BoundedTensorSpec(shape=(1,), dtype=tf.float32, name='observation', minimum=array(0., dtype=float32), maximum=array(3.4028235e+38, dtype=float32))
Reward Spec:
BoundedTensorSpec(shape=(1,), dtype=tf.int64, name='action', minimum=array(0, dtype=int64), maximum=array(18, dtype=int64))
Observation Spec:
BoundedTensorSpec(shape=(1,), dtype=tf.float32, name='observation', minimum=array(0., dtype=float32), maximum=array(3.4028235e+38, dtype=float32))
Reward Spec:
TensorSpec(shape=(), dtype=tf.float32, name='reward')


## import Q-Net

In [7]:
fc_layer_params = (128,)

q_net = q_network.QNetwork(
    filmEnv.observation_spec(),
    filmEnv.action_spec(),
    fc_layer_params=fc_layer_params)

## instantiate DqnAgent

In [8]:
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate)

train_step_counter = tf.Variable(0)

agent = dqn_agent.DqnAgent(
    filmEnv.time_step_spec(),
    filmEnv.action_spec(),
    q_network=q_net,
    optimizer=optimizer,
    td_errors_loss_fn=element_wise_squared_loss,
    train_step_counter=train_step_counter)

agent.initialize()

In [9]:

print('Observation Spec:')
print(filmEnv.time_step_spec().observation)
print('Action Spec:')
print(filmEnv.action_spec())

Observation Spec:
BoundedTensorSpec(shape=(1,), dtype=tf.float32, name='observation', minimum=array(0., dtype=float32), maximum=array(3.4028235e+38, dtype=float32))
Action Spec:
BoundedTensorSpec(shape=(1,), dtype=tf.int64, name='action', minimum=array(0, dtype=int64), maximum=array(18, dtype=int64))


## Policies

In [10]:
eval_policy = agent.policy
collect_policy = agent.collect_policy

random_policy = random_tf_policy.RandomTFPolicy(action_spec = filmEnv.action_spec(),
                                                time_step_spec = filmEnv.time_step_spec())

In [11]:
example_enviroment = tf_py_environment.TFPyEnvironment(FilmEnvironment(config_path='Zn.ini'))
time_step = example_enviroment.reset()
random_policy.action(time_step)

ValueError: Received a mix of batched and unbatched Tensors, or Tensors are not compatible with Specs.  num_outer_dims: 1.
Saw tensor_shapes:
   TimeStep(step_type=TensorShape([1]), reward=TensorShape([1]), discount=TensorShape([1]), observation=TensorShape([1, 1, 4]))
And spec_shapes:
   TimeStep(step_type=TensorShape([]), reward=TensorShape([]), discount=TensorShape([]), observation=TensorShape([1]))